# Vamos a hacer unos modelos para la predicción de energía.  
Esto es casi igual que lo de la clasificación pero tenemos que hacerlo con las labels de energia.

In [1]:
#importamos librerias

import sys
sys.path.append('../src/CTA-data-analisis-library/')
import os 
import subprocess
from datetime import datetime
import numpy as np 
import glob
import matplotlib.pyplot as plt
import tensorflow as tf 
import psutil
import re
import random
import shutil
import pickle
from numba import cuda
import gc

#propias
import unzipdata_and_first_treatments as manipulate
import loaddata4use
import model_creation_functions as models

In [2]:
tf.config.threading.set_inter_op_parallelism_threads(4)

In [6]:
#enviroment variables
npy_final_dir="../datos/elementos_npy"
base_dir_elementos="../datos/elementos"
elements=['gamma', 'electron']

In [7]:
def runs_disponibles(npy_dir,elements):
    lista=[]
    for i in elements:
        npy_element_dir=os.path.join(npy_dir,"npy_"+i)
        runs=[int(re.search("run_([0-9]{3})_0\.npy",i).group(1)) for i in os.listdir(npy_element_dir) if re.search("run_([0-9]{3})_0\.npy",i)]
        lista.append(runs)
    return lista


def new_create_main_list_runs(number_runs_per_element,posibles_runs):
    #esto es aleatorio por defecto, porque es lo unico que necesito por ahora    
    final=[]
    for ind,lista_runs_element in enumerate(posibles_runs):
        final.append(random.sample(lista_runs_element,number_runs_per_element[ind]))
    return final

#tenemos que hacer un ligero cambio porque se estan cargando con los ejes cambiados
def cambiar_ejes_lista(lista):
    for i,j in enumerate(lista):
        lista[i]=np.swapaxes(j,1,2)
    return lista


In [8]:
chose_runs=runs_disponibles(npy_final_dir,elements)
chose_runs[0].remove(2)
chose_runs[0].remove(2)
chose_runs[0].remove(3)
chose_runs[0].remove(3)
chose_runs[0].remove(3)

In [9]:
opciones_filtros=[
    [[12,16],[32,64],[64,12]],
    [[12,16,32],[64,128],[128,64,32]],
    [[64,128],[128,254,64],[32]]]

In [11]:
#vamos a llevar un registros del punto en el que se esta quedando el modelo
#voy a hacer una callback que guarde el último modelo, y con ello
#vamos a ir guardando los ultimo datos de la ejecucion y si hay un problema, retornamos a ese punto

#si tenemos que retomar el entrenamiento, vamos a indicar que modelo usar

device = cuda.get_current_device()
file_number="002"
n=1 #repes de boostrap
#primer bucle para arquitecturas
for i,arch in enumerate(opciones_filtros):
    print(f"{i}: {arch} \n")

    modelo=models.model_multi_tel_energy(len_inputs=4,input_shapes=[(55,93,1)],learning_rate=1e-4,filtros=arch,last_dense=[20,10])
    modelo.compile(optimizer="adam",loss="mse",metrics=["mae"])
    with open(f"../automat/logs/{file_number}_data_control_energy.txt","a") as registro:
        registro.write(f"Con arquitectura: {arch}, y uso de CPU {tf.config.experimental.get_memory_info('CPU:0')['current']>>20}Mb \n")

    #segundo_bucle para boostrap
    for k in range(n):
        print(f"\n Boostrap {k+1} de {n}, y uso de CPU {tf.config.experimental.get_memory_info('CPU:0')['current']>>20}Mb \n")

        list_runs=new_create_main_list_runs([2,11],chose_runs)#new_create_main_list_runs([2,6,6,6,6,6,6],chose_runs)
        with open(f"../automat/logs/{file_number}_data_control_energy.txt","a") as registro:
            registro.write(f"Boostrap {k+1} de {n},runs: {list_runs}, y uso de CPU {tf.config.experimental.get_memory_info('CPU:0')['current']>>20}Mb \n")
        x_train_list,x_test_list,y_train_list,y_test_list=loaddata4use.load_dataset_energy(npy_final_dir,base_dir_elementos,elementos=['gamma', 'electron'],main_list_runs=list_runs,telescopios=[1,2,3,4],test_size=0.1,same_quant="same",verbose=True,fill=True)

        x_train_list=cambiar_ejes_lista(x_train_list)
        x_test_list=cambiar_ejes_lista(x_test_list)

        
        hist=modelo.fit(x=x_train_list,y=y_train_list,epochs=1, validation_data=(x_test_list,y_test_list),batch_size=64)
        del x_train_list,x_test_list,y_train_list,y_test_list
        gc.collect()
        modelo.save(f"../modelos/{file_number}_modelo_filtro_{i}_en_boostrap_stage_{k+1}_energy.h5")
        with open(f"../modelos/performances/{file_number}_history_modelo_filtro_{i}_en_boostrap_stage_{k+1}_energy.pickle","wb") as pick:
            pickle.dump(hist,pick)    
    del modelo 
    gc.collect()
    tf.keras.backend.clear_session()
    device.reset()
        


0: [[12, 16], [32, 64], [64, 12]] 


 Boostrap 1 de 1, y uso de CPU 465Mb 

Load of names and common events
_______________
Element:  gamma  , Runs:  [15, 38]  Shape of common events (common events, energies):  (4, 1176) (4, 1176)
Element:  gamma  , Runs:  [15, 38]  Shape of common events (common events, energies):  (4, 1147) (4, 1147)
Element:  electron  , Runs:  [140, 108, 114, 129, 128, 55, 29, 17, 30, 56, 73]  Shape of common events (common events, energies):  (4, 239) (4, 239)
Element:  electron  , Runs:  [140, 108, 114, 129, 128, 55, 29, 17, 30, 56, 73]  Shape of common events (common events, energies):  (4, 252) (4, 252)
Element:  electron  , Runs:  [140, 108, 114, 129, 128, 55, 29, 17, 30, 56, 73]  Shape of common events (common events, energies):  (4, 246) (4, 246)
Element:  electron  , Runs:  [140, 108, 114, 129, 128, 55, 29, 17, 30, 56, 73]  Shape of common events (common events, energies):  (4, 228) (4, 228)
Element:  electron  , Runs:  [140, 108, 114, 129, 128, 55, 29, 17,

INFO:tensorflow:Assets written to: ram://c2623825-2c7f-459f-bb4f-65931bd3aeb5/assets


INFO:tensorflow:Assets written to: ram://c2623825-2c7f-459f-bb4f-65931bd3aeb5/assets


1: [[12, 16, 32], [64, 128], [128, 64, 32]] 


 Boostrap 1 de 1, y uso de CPU 468Mb 

Load of names and common events
_______________
Element:  gamma  , Runs:  [43, 49]  Shape of common events (common events, energies):  (4, 1094) (4, 1094)
Element:  gamma  , Runs:  [43, 49]  Shape of common events (common events, energies):  (4, 1100) (4, 1100)
Element:  electron  , Runs:  [1, 83, 143, 145, 156, 63, 136, 7, 44, 7, 8]  Shape of common events (common events, energies):  (4, 263) (4, 263)
Element:  electron  , Runs:  [1, 83, 143, 145, 156, 63, 136, 7, 44, 7, 8]  Shape of common events (common events, energies):  (4, 246) (4, 246)
Element:  electron  , Runs:  [1, 83, 143, 145, 156, 63, 136, 7, 44, 7, 8]  Shape of common events (common events, energies):  (4, 250) (4, 250)
Element:  electron  , Runs:  [1, 83, 143, 145, 156, 63, 136, 7, 44, 7, 8]  Shape of common events (common events, energies):  (4, 232) (4, 232)
Element:  electron  , Runs:  [1, 83, 143, 145, 156, 63, 136, 7, 44, 7, 8]  S

KeyboardInterrupt: 